# Utilizando Assistantes OpenAI

#### Imports e API key

In [2]:
import openai
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

client = openai.Client()

# Objetivo e vantagens Assistant

#### Permitem criar assistentes virtuais personalizados para tarefas específicas, como atendimento ao cliente, suporte técnico e automação de processos.

## Vantagens
##### Personalização: Ajuste o assistente ao tom e às necessidades do seu negócio.
#####  Relevância: Focado em tarefas e contextos específicos, fornecendo respostas e soluções mais precisas.
##### Controle Total: Permite ajustar e otimizar continuamente o comportamento e as respostas do assistente conforme as políticas e objetivos da empresa.


# Tipos de Assistant

### Code Interpreter:

Permite escrever, interpretar e depurar código em várias linguagens de programação. Além de fornecer suporte técnico e tutoria em programação, ele é utilizado para uma ampla gama de tarefas, como análise de dados, automação de processos e desenvolvimento de aplicativos. Sua capacidade de executar operações complexas diretamente no código o torna uma ferramenta poderosa para resolver problemas e desenvolver soluções personalizadas em diversos domínios.

### File Search:

Ferramenta que permite que assistente encontre informações em documentos externos, como arquivos CSV, PDFs, etc. Ele analisa esses documentos e guarda as informações , quando um usuário faz uma pergunta, o File Search busca nos documentos por informações relevantes para responder à pergunta de forma precisa. É como ter uma biblioteca de informações disponível para o assistente, ajudando-o a responder melhor às perguntas dos usuários.


### Function Calling

É como um controle remoto que permite ao assistente acionar ações específicas em outros programas ou sistemas. Por exemplo, se você tem um sistema de vendas e o assistente precisa atualizar o estoque de um produto, ele pode "chamar" a função responsável por isso no sistema de vendas. É como se o assistente desse um comando para que uma tarefa seja realizada em outro lugar. Isso permite que o assistente faça mais do que apenas responder perguntas, ele pode interagir diretamente com outros sistemas para realizar tarefas específicas.

# Criando o assistant

In [3]:
assitant = client.beta.assistants.create(
    name="Tutor de Matemática da Asimov",
    instructions='Você é um tutor pessoal de matemática da empresa Asimov. \
        Escreva e execute códigos para responder as perguntas de matemática que lhe forem passadas.',
    tools=[{'type': 'code_interpreter'}],  # Existem tools ja predefinidos, alem de poder criar um personalizado
    model='gpt-3.5-turbo-0125'
)

## Cria uma thread

In [4]:
thread = client.beta.threads.create()

## Adiciona mensagem a thread criada

In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role='user',
    content='Se eu jogar um dado honesto 1000 vezes, qual é a probabilidade de eu obter exatamente 150 vezes o número 6? Resolva com um código'
)

## Roda a thread no assistant

#### Posso passar instruções especificas para essa thread

In [6]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assitant.id,
    instructions='O nome do usuário é Adriano Soares e ele é um usuário Premium.'  # Se tivesse privilegios Premuim, concederia.
)

## Aguarda a thread rodar

#### Loop para verificar se a thread ja finalizou ou ainda está em progresso

#### Se finalizou, vai parar de rodar

In [7]:
import time

while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(
        thread_id=thread.id,
        run_id=run.id
    )


In [8]:
#Verificando status

run.status

'completed'

# Capturando e imprimindo a resposta

In [10]:
if run.status == 'completed':
    mensagens = client.beta.threads.messages.list(
        thread_id=thread.id
    )
    print(mensagens)
else:
    print('Erro', run.status)

SyncCursorPage[Message](data=[Message(id='msg_WKfADGPRCI1XFH8ZZ926iiVU', assistant_id='asst_JY0og5hAYSTtPm0jlsqOTQpU', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='A probabilidade de obter exatamente 150 vezes o número 6 ao jogar um dado honesto 1000 vezes é de aproximadamente 1.26%.\n\nSe tiver mais alguma dúvida ou se precisar de mais alguma informação, estou à disposição!'), type='text')], created_at=1716666397, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_TtX7l1GXSrdVJYKyjdoKDVwi', status=None, thread_id='thread_ibw4QHyL1ymitnRJK4yTTuOB'), Message(id='msg_UH9f8eFxT1dpgpXQWU8spRb9', assistant_id='asst_JY0og5hAYSTtPm0jlsqOTQpU', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Para calcular a probabilidade de obter exatamente 150 vezes o número 6 ao jogar um dado honesto 1000 vezes, podemos usar a distribuição binomial. A fó

In [9]:
print(mensagens.data[0].content[0].text.value)

A probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes é aproximadamente 0.0126, ou seja, cerca de 1.26%.


# Analisando os passos do modelo

In [ ]:
# Lista com todos os passos

run_steps = client.beta.threads.runs.steps.list(
  thread_id=thread.id,
  run_id=run.id
)

## Criando uma for para iterar sobre cada passo

###  Se o passo for tool_calls, detalhe-o.

### Se for message_creation, imprima a mensagem.

In [16]:
for step in run_steps.data[::-1]:
    print('\n=== Step:', step.step_details.type)
    if step.step_details.type == 'tool_calls':
        for tool_call in step.step_details.tool_calls:
            print('-----')
            print(tool_call.code_interpreter.input)
            print('-----')
            print('Result')
            print(tool_call.code_interpreter.outputs[0].logs)
            
    if step.step_details.type == 'message_creation':
        message = client.beta.threads.messages.retrieve(
            thread_id=thread.id,
            message_id=step.step_details.message_creation.message_id
        )
        print(message.content[0].text.value)
    


=== Step: message_creation
Para calcular a probabilidade de obter exatamente 150 vezes o número 6 ao lançar um dado honesto 1000 vezes, podemos usar a distribuição binomial. A fórmula para a probabilidade em uma distribuição binomial é dada por:

\[ P(X = k) = \binom{n}{k} \times p^k \times q^{n-k} \]

onde:
- \( n \) é o número total de tentativas (lançamentos do dado),
- \( k \) é o número de sucessos desejado (número 6),
- \( p \) é a probabilidade de sucesso em uma única tentativa,
- \( q \) é a probabilidade de fracasso em uma única tentativa (1 - p).

Neste caso, \( n = 1000 \) (número total de lançamentos do dado), \( k = 150 \) (número de vezes em que queremos obter o número 6), \( p = \frac{1}{6} \) (probabilidade de obter o número 6 em um único lançamento) e \( q = 1 - p \).

Vamos calcular essa probabilidade com um código Python:

=== Step: tool_calls
-----
from math import comb

# Definindo os parâmetros
n = 1000  # número total de lançamentos do dado
k = 150   # número de

### Podemos ver que ele executou um codigo em python que resolveu e deu a resposta.

In [11]:
mensagens

SyncCursorPage[Message](data=[Message(id='msg_WKfADGPRCI1XFH8ZZ926iiVU', assistant_id='asst_JY0og5hAYSTtPm0jlsqOTQpU', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='A probabilidade de obter exatamente 150 vezes o número 6 ao jogar um dado honesto 1000 vezes é de aproximadamente 1.26%.\n\nSe tiver mais alguma dúvida ou se precisar de mais alguma informação, estou à disposição!'), type='text')], created_at=1716666397, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_TtX7l1GXSrdVJYKyjdoKDVwi', status=None, thread_id='thread_ibw4QHyL1ymitnRJK4yTTuOB'), Message(id='msg_UH9f8eFxT1dpgpXQWU8spRb9', assistant_id='asst_JY0og5hAYSTtPm0jlsqOTQpU', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='Para calcular a probabilidade de obter exatamente 150 vezes o número 6 ao jogar um dado honesto 1000 vezes, podemos usar a distribuição binomial. A fó

In [16]:
mensagens.data[0]

Message(id='msg_WKfADGPRCI1XFH8ZZ926iiVU', assistant_id='asst_JY0og5hAYSTtPm0jlsqOTQpU', attachments=[], completed_at=None, content=[TextContentBlock(text=Text(annotations=[], value='A probabilidade de obter exatamente 150 vezes o número 6 ao jogar um dado honesto 1000 vezes é de aproximadamente 1.26%.\n\nSe tiver mais alguma dúvida ou se precisar de mais alguma informação, estou à disposição!'), type='text')], created_at=1716666397, incomplete_at=None, incomplete_details=None, metadata={}, object='thread.message', role='assistant', run_id='run_TtX7l1GXSrdVJYKyjdoKDVwi', status=None, thread_id='thread_ibw4QHyL1ymitnRJK4yTTuOB')